In [ ]:
# 初始化
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv
import json

_ = load_dotenv(find_dotenv())

client = OpenAI()


: 

In [ ]:
def print_json(data):
    """
    打印参数。如果参数是有结构的（如字典或列表），则以格式化的 JSON 形式打印；
    否则，直接打印该值。
    """
    if hasattr(data, 'model_dump_json'):
        data = json.loads(data.model_dump_json())

    if (isinstance(data, (list))):
        for item in data:
            print_json(item)
    elif (isinstance(data, (dict))):
        print(json.dumps(
            data,
            indent=4,
            ensure_ascii=False
        ))
    else:
        print(data)

: 

In [ ]:
def get_sql_completion(messages, model="gpt-3.5-turbo"):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0,
        tools=[{  # 摘自 OpenAI 官方示例 https://github.com/openai/openai-cookbook/blob/main/examples/How_to_call_functions_with_chat_models.ipynb
            "type": "function",
            "function": {
                "name": "ask_database",
                "description": "Use this function to answer user questions about business. \
                            Output should be a fully formed SQL query.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "query": {
                            "type": "string",
                            "description": f"""
                            SQL query extracting info to answer the user's question.
                            SQL should be written using this database schema:
                            {database_schema_string}
                            The query should be returned in plain text, not in JSON.
                            The query should only contain grammars supported by SQLite.
                            """,
                        }
                    },
                    "required": ["query"],
                }
            }
        }],
    )
    return response.choices[0].message

: 

In [ ]:
database_schema_string = """
CREATE TABLE orders (
    id INT PRIMARY KEY NOT NULL, -- 主键，不允许为空
    plan_id VARCHAR(255) NOT NULL, -- 套餐ID，不允许为空
    plan_name VARCHAR(255) NOT NULL, -- 套餐名称，不允许为空
    price_cny DECIMAL(10,2) NOT NULL, -- 人民币价格，不允许为空
    data_g INT NOT NULL, -- 流量_单位G，不允许为空
    requirement VARCHAR(255) -- 购买资格，可以为空
);
"""

: 

In [ ]:
import sqlite3

# 创建数据库连接
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# 创建orders表
cursor.execute(database_schema_string)

# 插入5条明确的模拟记录
# 经济套餐：月费50元，月流量10G
# 畅游套餐：月费180元，月流量100G
# 无限套餐：月费300元，月流量1000G
# 校园套餐：月费150元，月流量200G，限在校学生办理
mock_data = [
    (1, '1001', '经济套餐', 50.00, 10, None),             # 经济套餐
    (2, '1002', '畅游套餐', 180.00, 100, None),            # 畅游套餐
    (3, '1003', '无限套餐', 300.00, 1000, None),           # 无限套餐
    (4, '1004', '校园套餐', 150.00, 200, '在校学生')       # 校园套餐
]

for record in mock_data:
    cursor.execute('''
    INSERT INTO orders (id, plan_id, plan_name, price_cny, data_g, requirement)
    VALUES (?, ?, ?, ?, ?, ?)
    ''', record)
# 提交事务
conn.commit()

: 

In [ ]:
def ask_database(query):
    cursor.execute(query)
    records = cursor.fetchall()
    return records



prompt = "有没有土豪套餐"
prompt = "办个200G的套餐"
prompt = "有没有流量大的套餐"
prompt = "200元以下，流量大的套餐有啥"
prompt = "你说那个10G的套餐，叫啥名字"

messages = [
    {"role": "system", "content": "你是一个手机数据套餐销售客服，基于数据库的数据回答问题"},
    {"role": "user", "content": prompt}
]
response = get_sql_completion(messages)
if response.content is None:
    response.content = ""
messages.append(response)
print("====Function Calling====")
print_json(response)

if response.tool_calls is not None:
    tool_call = response.tool_calls[0]
    if tool_call.function.name == "ask_database":
        arguments = tool_call.function.arguments
        args = json.loads(arguments)
        print("====SQL====")
        print(args["query"])
        result = ask_database(args["query"])
        print("====DB Records====")
        print(result)

        messages.append({
            "tool_call_id": tool_call.id,
            "role": "tool",
            "name": "ask_database",
            "content": str(result)
        })
        response = get_sql_completion(messages)
        print("====最终回复====")
        print(response.content)

print("=====对话历史=====")
print_json(messages)

: 